In [1]:
import torch
import math
from flash_attn import flash_attn_func

In [30]:
bsz=1
seq_len=4096
dim=128
num_heads=4
dtype = torch.bfloat16

q = torch.rand((bsz, seq_len, num_heads, dim)).to(dtype).cuda().transpose(1, 2)
k = torch.rand((bsz, seq_len, num_heads, dim)).to(dtype).cuda().transpose(1, 2)
v = torch.rand((bsz, seq_len, num_heads, dim)).to(dtype).cuda().transpose(1, 2)

# flash_attn
ret1 = flash_attn_func(q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2), causal=True).transpose(1, 2)

# scaled_dot_product_attention
ret2 = torch.nn.functional.scaled_dot_product_attention(q, k, v, is_causal=True)

# manual
attn_weights = torch.matmul(q, k.transpose(2,3))/math.sqrt(dim)
mask = torch.full((seq_len, seq_len), torch.finfo(dtype).min).cuda()
mask = torch.triu(mask, 1)
attn_weights = attn_weights + mask[None, None, :, :]
attn_weights = torch.nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(dtype)
ret3 = torch.matmul(attn_weights, v)


print('Flash == SDPA:', torch.allclose(ret1, ret2, atol=1e-2))
print('Flash == Manual:', torch.allclose(ret1, ret3, atol=1e-2))
print('SDPA == Manual:', torch.allclose(ret2, ret3, atol=1e-2))

Flash == SDPA: True
Flash == Manual: True
SDPA == Manual: True


In [19]:
ret1[0,0,0]

tensor([0.0854, 0.9961, 0.1338, 0.1050, 0.7812, 0.4980, 0.9219, 0.5977, 0.8672,
        0.6172, 0.8008, 0.0532, 0.0815, 0.0564, 0.5195, 0.7148, 0.0840, 0.9141,
        0.7266, 0.0175, 0.0845, 0.5430, 0.5508, 0.1992, 0.7734, 0.3281, 0.3555,
        0.0466, 0.0032, 0.3691, 0.6094, 0.0830, 0.7656, 0.2598, 0.0820, 0.8086,
        0.0811, 0.2158, 0.3320, 0.8242, 0.8008, 0.0108, 0.4902, 0.5898, 0.5898,
        0.6016, 0.4297, 0.2695, 0.5156, 0.7422, 0.5156, 0.1387, 0.6758, 0.4102,
        0.4141, 0.3613, 0.8672, 0.2080, 0.1709, 0.0747, 0.7266, 0.7227, 0.3418,
        0.4102, 0.8438, 0.3906, 0.8438, 0.5625, 0.4375, 0.8828, 0.9375, 0.9648,
        0.4199, 1.0000, 0.0613, 0.6836, 0.1318, 0.9805, 0.0889, 0.9023, 0.7852,
        0.8672, 0.9727, 0.7305, 0.2598, 0.2832, 0.4688, 0.2676, 0.4160, 0.6328,
        0.1631, 0.0092, 0.9219, 0.5234, 0.6016, 0.4961, 0.0125, 0.3965, 0.0698,
        0.0051, 0.2490, 0.4238, 0.2773, 0.1143, 0.1592, 0.5391, 0.4824, 0.7539,
        0.5156, 0.5469, 0.3066, 0.4902, 

In [20]:
ret2[0,0,0]

tensor([0.0854, 0.9961, 0.1338, 0.1050, 0.7812, 0.4980, 0.9219, 0.5977, 0.8672,
        0.6172, 0.8008, 0.0532, 0.0815, 0.0564, 0.5195, 0.7148, 0.0840, 0.9141,
        0.7266, 0.0175, 0.0845, 0.5430, 0.5508, 0.1992, 0.7734, 0.3281, 0.3555,
        0.0466, 0.0032, 0.3691, 0.6094, 0.0830, 0.7656, 0.2598, 0.0820, 0.8086,
        0.0811, 0.2158, 0.3320, 0.8242, 0.8008, 0.0108, 0.4902, 0.5898, 0.5898,
        0.6016, 0.4297, 0.2695, 0.5156, 0.7422, 0.5156, 0.1387, 0.6758, 0.4102,
        0.4141, 0.3613, 0.8672, 0.2080, 0.1709, 0.0747, 0.7266, 0.7227, 0.3418,
        0.4102, 0.8438, 0.3906, 0.8438, 0.5625, 0.4375, 0.8828, 0.9375, 0.9648,
        0.4199, 1.0000, 0.0613, 0.6836, 0.1318, 0.9805, 0.0889, 0.9023, 0.7852,
        0.8672, 0.9727, 0.7305, 0.2598, 0.2832, 0.4688, 0.2676, 0.4160, 0.6328,
        0.1631, 0.0092, 0.9219, 0.5234, 0.6016, 0.4961, 0.0125, 0.3965, 0.0698,
        0.0051, 0.2490, 0.4238, 0.2773, 0.1143, 0.1592, 0.5391, 0.4824, 0.7539,
        0.5156, 0.5469, 0.3066, 0.4902, 

In [21]:
ret3[0, 0, 0]

tensor([0.0854, 0.9961, 0.1338, 0.1050, 0.7812, 0.4980, 0.9219, 0.5977, 0.8672,
        0.6172, 0.8008, 0.0532, 0.0815, 0.0564, 0.5195, 0.7148, 0.0840, 0.9141,
        0.7266, 0.0175, 0.0845, 0.5430, 0.5508, 0.1992, 0.7734, 0.3281, 0.3555,
        0.0466, 0.0032, 0.3691, 0.6094, 0.0830, 0.7656, 0.2598, 0.0820, 0.8086,
        0.0811, 0.2158, 0.3320, 0.8242, 0.8008, 0.0108, 0.4902, 0.5898, 0.5898,
        0.6016, 0.4297, 0.2695, 0.5156, 0.7422, 0.5156, 0.1387, 0.6758, 0.4102,
        0.4141, 0.3613, 0.8672, 0.2080, 0.1709, 0.0747, 0.7266, 0.7227, 0.3418,
        0.4102, 0.8438, 0.3906, 0.8438, 0.5625, 0.4375, 0.8828, 0.9375, 0.9648,
        0.4199, 1.0000, 0.0613, 0.6836, 0.1318, 0.9805, 0.0889, 0.9023, 0.7852,
        0.8672, 0.9727, 0.7305, 0.2598, 0.2832, 0.4688, 0.2676, 0.4160, 0.6328,
        0.1631, 0.0092, 0.9219, 0.5234, 0.6016, 0.4961, 0.0125, 0.3965, 0.0698,
        0.0051, 0.2490, 0.4238, 0.2773, 0.1143, 0.1592, 0.5391, 0.4824, 0.7539,
        0.5156, 0.5469, 0.3066, 0.4902, 

torch.Size([1, 4, 4096, 128])